In [1]:
import pandas as pd
import os
import numpy as np
import time
import shutil

directory = "data/simulacrum_v2.1.0/Data"
output = "data/simulacrum_v2.1.0/Processed"
tables = {
    "sim_av_gene.csv": {
        "GENEID": "int64",
        "TUMOURID": "int64",
        "PATIENTID": "int64",
        "GENE_DESC": "category",
        "GENE": "int64",
        "COUNT_TESTS": "int64",
        "COUNT_RESULTS": "int64",
        "COUNT_DATE": "int64",
        "ALL_TESTSTATUSES": "category",
        "OVERALL_TS": "category",
        "NO_OF_AB_GATS": "int64",
        "DNASEQ_GAT": "category",
        "METHYL_GAT": "category",
        "EXP_GAT": "category",
        "COPYNO_GAT": "category",
        "FUS_TRANS_GAT": "category",
        "ABNORMAL_GAT": "category",
        "NO_OF_SEQ_VARS": "int64",
        "ALL_SEQ_VARS": "category",
        "SEQ_VAR": "category",
        "DATE_OVERALL_TS": "datetime",
        "MIN_DATE": "datetime",
        "MAX_DATE": "datetime",
        "ALL_PRO_IMPS": "category",
        "NO_OF_PRO_IMPS": "int64",
        "PRO_IMP": "category",
        "METHODS": "category",
        "LAB_NAME": "category",
        "BEST_DATE_SOURCE_OVERALL_TS": "category",
    },
    "sim_av_patient.csv": {
        "PATIENTID": "int64",
        "GENDER": "category",
        "ETHNICITY": "category",
        "DEATHCAUSECODE_1A": "category",
        "DEATHCAUSECODE_1B": "category",
        "DEATHCAUSECODE_1C": "category",
        "DEATHCAUSECODE_2": "category",
        "DEATHCAUSECODE_UNDERLYING": "category",
        "DEATHLOCATIONCODE": "category",
        "VITALSTATUS": "category",
        "VITALSTATUSDATE": "datetime",
        "LINKNUMBER": "int64",
    },
    "sim_av_tumour.csv": {
        "TUMOURID": "int64",
        "GENDER": "category",
        "PATIENTID": "int64",
        "DIAGNOSISDATEBEST": "datetime",
        "SITE_ICD10_O2_3CHAR": "category",
        "SITE_ICD10_O2": "category",
        "SITE_ICD10R4_O2_FROM2013": "category",
        "SITE_ICDO3REV2011": "category",
        "SITE_ICDO3REV2011_3CHAR": "category",
        "SITE_ICD10R4_O2_3CHAR_FROM2013": "category",
        "MORPH_ICD10_O2": "int64",
        "MORPH_ICDO3REV2011": "Int64",
        "BEHAVIOUR_ICD10_O2": "category",
        "BEHAVIOUR_ICDO3REV2011": "category",
        "T_BEST": "category",
        "N_BEST": "category",
        "M_BEST": "category",
        "STAGE_BEST": "category",
        "GRADE": "category",
        "AGE": "category",
        "CREG_CODE": "category",
        "STAGE_BEST_SYSTEM": "category",
        "LATERALITY": "category",
        "SCREENINGSTATUSFULL_CODE": "category",
        "ER_STATUS": "category",
        "PR_STATUS": "category",
        "HER2_STATUS": "category",
        "QUINTILE_2019": "category",
        "DATE_FIRST_SURGERY": "datetime",
        "CANCERCAREPLANINTENT": "category",
        "PERFORMANCESTATUS": "category",
        "CHRL_TOT_27_03": "category",
        "COMORBIDITIES_27_03": "category",
        "GLEASON_PRIMARY": "category",
        "GLEASON_SECONDARY": "category",
        "GLEASON_TERTIARY": "category",
        "GLEASON_COMBINED": "category",
    },
    "sim_rtds_combined.csv": {
        "PATIENTID": "int64",
        "PRESCRIPTIONID": "int64",
        "RTTREATMENTMODALITY": "category",
        "RADIOTHERAPYPRIORITY": "category",
        "RADIOTHERAPYINTENT": "category",
        "RTPRESCRIBEDDOSE": "category",
        "RTPRESCRIBEDFRACTIONS": "category",
        "RTACTUALDOSE": "category",
        "RTACTUALFRACTIONS": "category",
        "RTTREATMENTREGION": "category",
        "RTTREATMENTANATOMICALSITE": "category",
        "DECISIONTOTREATDATE": "datetime",
        "EARLIESTCLINAPPROPDATE": "datetime",
        "RADIOTHERAPYEPISODEID": "int64",
        "LINKCODE": "category",
        "RADIOISOTOPE": "category",
        "RADIOTHERAPYBEAMTYPE": "category",
        "RADIOTHERAPYBEAMENERGY": "category",
        "TIMEOFEXPOSURE": "time",
        "APPTDATE": "datetime",
        "ATTENDID": "special",
    },
    "sim_rtds_episode.csv": {
        "PATIENTID": "int64",
        "RADIOTHERAPYEPISODEID": "int64",
        "ATTENDID": "special",
        "APPTDATE": "datetime",
        "LINKCODE": "category",
        "DECISIONTOTREATDATE": "datetime",
        "EARLIESTCLINAPPROPDATE": "datetime",
        "RADIOTHERAPYPRIORITY": "category",
        "RADIOTHERAPYINTENT": "category",
    },
    "sim_rtds_exposure.csv": {
        "PRESCRIPTIONID": "int64",
        "RADIOISOTOPE": "category",
        "RADIOTHERAPYBEAMTYPE": "category",
        "RADIOTHERAPYBEAMENERGY": "category",
        "TIMEOFEXPOSURE": "time",
        "APPTDATE": "datetime",
        "ATTENDID": "special",
        "PATIENTID": "int64",
        "RADIOTHERAPYEPISODEID": "int64",
        "LINKCODE": "category",
    },
    "sim_rtds_prescription.csv": {
        "PATIENTID": "int64",
        "PRESCRIPTIONID": "int64",
        "RTTREATMENTMODALITY": "category",
        "RTPRESCRIBEDDOSE": "category",
        "RTPRESCRIBEDFRACTIONS": "category",
        "RTACTUALDOSE": "category",
        "RTACTUALFRACTIONS": "category",
        "RTTREATMENTREGION": "category",
        "RTTREATMENTANATOMICALSITE": "category",
        "RADIOTHERAPYEPISODEID": "int64",
        "LINKCODE": "category",
        "ATTENDID": "special",
        "APPTDATE": "datetime",
    },
    "sim_sact_cycle.csv": {
        "MERGED_REGIMEN_ID": "int64",
        "MERGED_CYCLE_ID": "int64",
        "CYCLE_NUMBER": "int64",
        "START_DATE_OF_CYCLE": "datetime",
        "OPCS_PROCUREMENT_CODE": "category",
        "PERF_STATUS_START_OF_CYCLE": "category",
    },
    "sim_sact_drug_detail.csv": {
        "MERGED_DRUG_DETAIL_ID": "int64",
        "MERGED_CYCLE_ID": "int64",
        "ACTUAL_DOSE_PER_ADMINISTRATION": "float64",
        "OPCS_DELIVERY_CODE": "category",
        "ADMINISTRATION_ROUTE": "category",
        "ADMINISTRATION_DATE": "datetime",
        "DRUG_GROUP": "category",
    },
    "sim_sact_outcome.csv": {
        "MERGED_REGIMEN_ID": "int64",
        "DATE_OF_FINAL_TREATMENT": "datetime",
        "REGIMEN_MOD_DOSE_REDUCTION": "category",
        "REGIMEN_MOD_TIME_DELAY": "category",
        "REGIMEN_MOD_STOPPED_EARLY": "category",
        "REGIMEN_OUTCOME_SUMMARY": "category",
    },
    "sim_sact_regimen.csv": {
        "ENCORE_PATIENT_ID": "int64",
        "MERGED_REGIMEN_ID": "int64",
        "HEIGHT_AT_START_OF_REGIMEN": "float64",
        "WEIGHT_AT_START_OF_REGIMEN": "float64",
        "INTENT_OF_TREATMENT": "category",
        "DATE_DECISION_TO_TREAT": "datetime",
        "START_DATE_OF_REGIMEN": "datetime",
        "MAPPED_REGIMEN": "category",
        "CLINICAL_TRIAL": "category",
        "CHEMO_RADIATION": "category",
        "BENCHMARK_GROUP": "category",
        "LINK_NUMBER": "int64",
    },
}

In [2]:
for t, dtypes in tables.items():
    print(f"\n\n######## Reading {t}")
    df = pd.read_csv(os.path.join(directory, t))
    r = df.iloc[0]
    for k, v in df.dtypes.items():
        print(f"{k:>25s}: {str(v):>10s} {len(df[k].unique())} {r[k]}")



######## Reading sim_av_gene.csv
                   GENEID:      int64 255728 10030620
                 TUMOURID:      int64 96494 10090743
                PATIENTID:      int64 94908 10403207
                GENE_DESC:     object 166 CD274 (PD-L1)
                     GENE:      int64 166 4842
              COUNT_TESTS:      int64 15 1
            COUNT_RESULTS:      int64 16 2
               COUNT_DATE:      int64 13 1
         ALL_TESTSTATUSES:     object 339 b:normal,b:normal
               OVERALL_TS:     object 7 b:normal
            NO_OF_AB_GATS:      int64 4 0
               DNASEQ_GAT:     object 5 n:not_applicable_null
               METHYL_GAT:     object 5 n:not_applicable_null
                  EXP_GAT:     object 7 b:normal_exp
               COPYNO_GAT:     object 6 n:not_applicable_null
            FUS_TRANS_GAT:     object 5 n:not_applicable_null
             ABNORMAL_GAT:     object 9 nan
           NO_OF_SEQ_VARS:      int64 6 0
             ALL_SEQ_VARS:     obje

/tmp/ipykernel_362906/709513949.py:3: DtypeWarning: Columns (14,15,16,17,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, t))


                 TUMOURID:      int64 1995570 10399610
                   GENDER:      int64 3 1
                PATIENTID:      int64 1871605 10000001
        DIAGNOSISDATEBEST:     object 1461 2017-03-31
      SITE_ICD10_O2_3CHAR:     object 109 C44
            SITE_ICD10_O2:     object 537 C444
SITE_ICD10R4_O2_3CHAR_FROM2013:     object 110 C44
 SITE_ICD10R4_O2_FROM2013:     object 569 C444
        SITE_ICDO3REV2011:     object 325 C444
  SITE_ICDO3REV2011_3CHAR:     object 71 C44
           MORPH_ICD10_O2:      int64 446 8070
       MORPH_ICDO3REV2011:    float64 615 8070.0
       BEHAVIOUR_ICD10_O2:      int64 7 3
   BEHAVIOUR_ICDO3REV2011:      int64 7 3
                   T_BEST:     object 40 nan
                   N_BEST:     object 22 nan
                   M_BEST:     object 11 nan
               STAGE_BEST:     object 99 ?
                    GRADE:     object 9 G3
                      AGE:      int64 108 84
                CREG_CODE:     object 8 L1201
        STAGE_BEST_

/tmp/ipykernel_362906/709513949.py:3: DtypeWarning: Columns (15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, t))


                PATIENTID:      int64 413169 10000008
           PRESCRIPTIONID:      int64 657648 10000001
      RTTREATMENTMODALITY:    float64 3 5.0
     RADIOTHERAPYPRIORITY:     object 5 R
       RADIOTHERAPYINTENT:    float64 4 2.0
         RTPRESCRIBEDDOSE:    float64 116 35.0
    RTPRESCRIBEDFRACTIONS:    float64 67 5.0
             RTACTUALDOSE:    float64 4893 35.0
        RTACTUALFRACTIONS:    float64 74 5.0
        RTTREATMENTREGION:     object 13 A
RTTREATMENTANATOMICALSITE:     object 481 Z501
      DECISIONTOTREATDATE:     object 4200 2018-07-28
   EARLIESTCLINAPPROPDATE:     object 3864 2018-07-28
    RADIOTHERAPYEPISODEID:      int64 656560 1181234
                 LINKCODE:     object 60 LKY
             RADIOISOTOPE:     object 27 nan
     RADIOTHERAPYBEAMTYPE:     object 9 T1
   RADIOTHERAPYBEAMENERGY:    float64 86 6.0
           TIMEOFEXPOSURE:     object 1058 10:09
                 APPTDATE:     object 3865 2018-08-11
                 ATTENDID:     object 5716677

...

           PRESCRIPTIONID:      int64 657648 10000001
             RADIOISOTOPE:     object 27 nan
     RADIOTHERAPYBEAMTYPE:     object 9 T1
   RADIOTHERAPYBEAMENERGY:    float64 86 6.0
           TIMEOFEXPOSURE:     object 1058 10:09
                 APPTDATE:     object 3865 2018-08-11
                 ATTENDID:     object 5812265 10000001.20180811
                PATIENTID:      int64 413169 10000008
    RADIOTHERAPYEPISODEID:      int64 656560 1181234
                 LINKCODE:     object 60 LKY


######## Reading sim_rtds_prescription.csv


/tmp/ipykernel_362906/709513949.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, t))


                PATIENTID:      int64 413169 10000008
           PRESCRIPTIONID:      int64 657648 10000001
      RTTREATMENTMODALITY:    float64 3 5.0
         RTPRESCRIBEDDOSE:    float64 116 35.0
    RTPRESCRIBEDFRACTIONS:    float64 67 5.0
             RTACTUALDOSE:    float64 4893 35.0
        RTACTUALFRACTIONS:    float64 74 5.0
        RTTREATMENTREGION:     object 13 A
RTTREATMENTANATOMICALSITE:     object 481 Z501
    RADIOTHERAPYEPISODEID:      int64 656560 1181234
                 LINKCODE:     object 60 LKY
                 ATTENDID:     object 5807529 10000001.20180811
                 APPTDATE:     object 3865 2018-08-11


######## Reading sim_sact_cycle.csv


/tmp/ipykernel_362906/709513949.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, t))


        MERGED_REGIMEN_ID:      int64 756595 10030621
          MERGED_CYCLE_ID:      int64 2741674 10000001
             CYCLE_NUMBER:      int64 225 1
      START_DATE_OF_CYCLE:     object 3746 2020-04-28
    OPCS_PROCUREMENT_CODE:     object 118 X713
PERF_STATUS_START_OF_CYCLE:     object 30 nan


######## Reading sim_sact_drug_detail.csv
    MERGED_DRUG_DETAIL_ID:      int64 7662030 10000001
          MERGED_CYCLE_ID:      int64 2729567 10000001
ACTUAL_DOSE_PER_ADMINISTRATION:    float64 5634 420.0
       OPCS_DELIVERY_CODE:     object 93 nan
     ADMINISTRATION_ROUTE:    float64 24 2.0
      ADMINISTRATION_DATE:     object 3758 2020-04-28
               DRUG_GROUP:     object 398 IBRUTINIB


######## Reading sim_sact_outcome.csv
        MERGED_REGIMEN_ID:      int64 784135 10030622
  DATE_OF_FINAL_TREATMENT:     object 3376 nan
REGIMEN_MOD_DOSE_REDUCTION:     object 5 N
   REGIMEN_MOD_TIME_DELAY:     object 8 nan
REGIMEN_MOD_STOPPED_EARLY:     object 7 nan
  REGIMEN_OUTCOME_SUMMAR

In [4]:
# try:
#     shutil.rmtree(output)
# except:
#     pass
os.makedirs(output, exist_ok=True)

skip = [
    # "sim_av_gene.csv",
    # "sim_av_patient.csv",
    # "sim_av_tumour.csv",
    # "sim_rtds_combined.csv",
    # "sim_rtds_episode.csv",
    # "sim_rtds_exposure.csv",
]

for t, dtypes in tables.items():
    if t in skip:
        continue
    print(f"######## Processing {t}")
    start = time.perf_counter()

    dates = [k for k, v in dtypes.items() if v == "datetime"]
    other = {
        k: v for k, v in dtypes.items() if v not in ("datetime", "time", "special")
    }  # drop times and attendid for now

    df = pd.read_csv(
        os.path.join(directory, t),
        dtype=other,
        parse_dates=dates,
        usecols=list(other.keys()) + dates,
    )
    df.to_parquet(os.path.join(output, t.replace(".csv", ".pq")), index=False)

    end = time.perf_counter()
    print(f"Processed {t:>25s} with {df.shape[0]:9_d} rows, {df.shape[1]:2d} cols, {df.memory_usage(deep=True).sum()//1e6:_.0f}MB in {end-start:5.2f} seconds")

######## Processing sim_av_gene.csv
Processed           sim_av_gene.csv with   255_728 rows, 29 cols, 47MB in  3.40 seconds
######## Processing sim_av_patient.csv
Processed        sim_av_patient.csv with 1_871_605 rows, 12 cols, 81MB in  7.50 seconds
######## Processing sim_av_tumour.csv
Processed         sim_av_tumour.csv with 1_995_570 rows, 37 cols, 168MB in 16.42 seconds
######## Processing sim_rtds_combined.csv
Processed     sim_rtds_combined.csv with 13_201_531 rows, 19 cols, 832MB in 111.16 seconds
######## Processing sim_rtds_episode.csv
Processed      sim_rtds_episode.csv with 5_843_642 rows,  8 cols, 251MB in 46.55 seconds
######## Processing sim_rtds_exposure.csv
Processed     sim_rtds_exposure.csv with 13_201_531 rows,  8 cols, 475MB in 44.81 seconds
######## Processing sim_rtds_prescription.csv
Processed sim_rtds_prescription.csv with 5_843_642 rows, 12 cols, 245MB in 22.14 seconds
######## Processing sim_sact_cycle.csv
Processed        sim_sact_cycle.csv with 2_741_674 ro